In [111]:
from bs4 import *
import requests
from pyquery import *
import pandas as pd
import numpy as np
import re


In [112]:
pq = PyQuery(url="https://www.espncricinfo.com/series/icc-cricket-world-cup-2023-24-1367856/india-vs-australia-final-1384439/full-scorecard")


In [113]:
full_tables_data = pq("table").text().split("\n")


In [114]:
india_batting_table_start_index = 0
india_batting_table_end_index = full_tables_data.index("Extras")
india_batting_table = full_tables_data[0:india_batting_table_end_index]
india_batting_table = np.array(india_batting_table[india_batting_table.index("SR")+1:])
no_of_rows = int(np.round(len(india_batting_table)/8,0))
india_batting_table = india_batting_table.reshape((no_of_rows,8))
india_batting = pd.DataFrame(india_batting_table)
india_batting.columns = ["Batter","Status","Runs","Balls","Minutes","Fours","Sixes","Strike Rate"]
india_batting.iloc[:,2:7] = india_batting.iloc[:,2:7].astype(dtype="int64")
india_batting["Strike Rate"] = india_batting["Strike Rate"].astype("float64")
india_batting


,Batter,Status,Runs,Balls,Minutes,Fours,Sixes,Strike Rate
0,Rohit Sharma (c),c Head b Maxwell,47,31,44,4,3,151.61
1,Shubman Gill,c Zampa b Starc,4,7,21,0,0,57.14
2,Virat Kohli,b Cummins,54,63,99,4,0,85.71
3,Shreyas Iyer,c †Inglis b Cummins,4,3,3,1,0,133.33
4,KL Rahul †,c †Inglis b Starc,66,107,133,1,0,61.68
5,Ravindra Jadeja,c †Inglis b Hazlewood,9,22,33,0,0,40.90
6,Suryakumar Yadav,c †Inglis b Hazlewood,18,28,57,1,0,64.28
7,Mohammed Shami,c †Inglis b Starc,6,10,9,1,0,60.00
8,Jasprit Bumrah,lbw b Zampa,1,3,5,0,0,33.33
9,Kuldeep Yadav,run out (Labuschagne/Cummins),10,18,28,0,0,55.55


In [115]:
remaining_tables_data_1 = full_tables_data[india_batting_table_end_index:]
india_extras_list = remaining_tables_data_1[1].split(",")
india_extras = dict()
for i in india_extras_list:
    india_extras["".join(re.findall(pattern="[A-Za-z]",string=i))] = int("".join(re.findall(pattern="\d",string=i)))
india_total = india_batting["Runs"].sum() + np.array([i for i in india_extras.values()]).sum()
india_total_balls = india_batting["Balls"].sum()
india_overs = int(np.floor(india_total_balls / 6))
india_balls = india_total_balls - india_overs * 6


In [118]:
remaining_tables_data_2 = remaining_tables_data_1[6:]
india_fall_of_wickets_list = remaining_tables_data_2[0]
india_fall_of_wickets_list_1 = india_fall_of_wickets_list[len("Fall of wickets: "):india_fall_of_wickets_list.index("\xa0•\xa0DRS BOWLING")].split("), ")
india_fall_of_wickets = pd.DataFrame(columns=["Wicket","Run","Batter","Over","Ball"])
fall = []
ctr = 0
for i in india_fall_of_wickets_list_1:
    wickets_runs = i[:i.index(" (")].split("-")
    india_fall_of_wickets.loc[ctr,"Wicket"] = wickets_runs[0]
    india_fall_of_wickets.loc[ctr,"Run"] = wickets_runs[1]
    india_fall_of_wickets.loc[ctr,"Batter"] = i[i.index(" (")+2:i.index(", ")]
    over = i[i.index(", ")+2:i.index(" ov")].split(".")
    india_fall_of_wickets.loc[ctr,"Over"] = int(over[0])
    india_fall_of_wickets.loc[ctr,"Ball"] = int(over[1])
    if india_fall_of_wickets.loc[ctr,"Ball"] == 6:
        india_fall_of_wickets.loc[ctr,"Over"] = india_fall_of_wickets.loc[ctr,"Over"] + 1
        india_fall_of_wickets.loc[ctr,"Ball"] = 0
    ctr += 1
india_fall_of_wickets


,Wicket,Run,Batter,Over,Ball
0,1,30,Shubman Gill,4,2
1,2,76,Rohit Sharma,9,4
2,3,81,Shreyas Iyer,10,2
3,4,148,Virat Kohli,28,3
4,5,178,Ravindra Jadeja,35,5
5,6,203,KL Rahul,41,3
6,7,211,Mohammed Shami,43,4
7,8,214,Jasprit Bumrah,44,5
8,9,226,Suryakumar Yadav,47,3
9,10,240,Kuldeep Yadav,50,0


In [135]:
remaining_tables_data_3 = remaining_tables_data_2[remaining_tables_data_2.index("NB")+1:remaining_tables_data_2.index("0 BATTING")+1]
remaining_tables_data_4 = remaining_tables_data_2[remaining_tables_data_2.index("0 BATTING"):]

ctr = 0
dts = []
dt = []
ctr = 0
for i in remaining_tables_data_3:
    dt.append(i)
    if i=="See all photos":
        dts.append(dt)
        dt = []
    ctr += 1
dts.append(dt)

australia_bowling_list = []
for i in dts[:-1]:
    australia_bowling_list.append(i[0:11])

australia_bowling_list.append(dts[-1][:dts[-1].index("Travis Head")])

kk = dts[-1][dts[-1].index("Travis Head"):-1]
kl = [str(dts[-1][-1].split(" ")[0])]
kk = kk + kl

australia_bowling_list.append(kk)

australia_bowling = pd.DataFrame(australia_bowling_list,columns=["Bowler","Overs","Maidens","Runs","Wickets","Economy","Zeros","Fours","Sixes","Wides","No Balls"])
australia_bowling


,Bowler,Overs,Maidens,Runs,Wickets,Economy,Zeros,Fours,Sixes,Wides,No Balls
0,Mitchell Starc,10,0,55,3,5.50,30,4,1,3,0
1,Josh Hazlewood,10,0,60,2,6.00,22,4,1,1,0
2,Glenn Maxwell,6,0,35,1,5.83,19,4,1,0,0
3,Pat Cummins,10,0,34,2,3.40,30,0,0,2,0
4,Adam Zampa,10,0,44,1,4.40,22,1,0,1,0
5,Mitchell Marsh,2,0,5,0,2.50,7,0,0,0,0
6,Travis Head,2,0,4,0,2.00,8,0,0,0,0


In [136]:
australia_batting_table_start_index = 0
australia_batting_table_end_index = remaining_tables_data_4.index("Extras")
australia_batting_table = remaining_tables_data_4[0:australia_batting_table_end_index]
australia_batting_table = np.array(australia_batting_table[australia_batting_table.index("SR")+1:])
no_of_rows = int(np.round(len(australia_batting_table)/8,0))
australia_batting_table = australia_batting_table.reshape((no_of_rows,8))
australia_batting = pd.DataFrame(australia_batting_table)
australia_batting.columns = ["Batter","Status","Runs","Balls","Minutes","Fours","Sixes","Strike Rate"]
australia_batting.iloc[:,2:7] = australia_batting.iloc[:,2:7].astype(dtype="int64")
australia_batting["Strike Rate"] = australia_batting["Strike Rate"].astype("float64")
australia_batting


,Batter,Status,Runs,Balls,Minutes,Fours,Sixes,Strike Rate
0,David Warner,c Kohli b Mohammed Shami,7,3,6,1,0,233.33
1,Travis Head,c Shubman Gill b Mohammed Siraj,137,120,166,15,4,114.16
2,Mitchell Marsh,c †Rahul b Bumrah,15,15,15,1,1,100.00
3,Steven Smith,lbw b Bumrah,4,9,11,1,0,44.44
4,Marnus Labuschagne,not out,58,110,133,4,0,52.72
5,Glenn Maxwell,not out,2,1,1,0,0,200.00


In [137]:
remaining_tables_data_5 = remaining_tables_data_4[australia_batting_table_end_index:]
australia_extras_list = remaining_tables_data_5[1].split(",")
australia_extras = dict()
for i in australia_extras_list:
    australia_extras["".join(re.findall(pattern="[A-Za-z]",string=i))] = int("".join(re.findall(pattern="\d",string=i)))
australia_did_not_bat = remaining_tables_data_5[remaining_tables_data_5.index("Did not bat:")+1:remaining_tables_data_5.index("O")]
adnb = australia_did_not_bat[-1]
australia_did_not_bat = australia_did_not_bat[:-1]
remaining_tables_data_6 = remaining_tables_data_5[remaining_tables_data_5.index(adnb):]


In [138]:
australia_fall_of_wickets_list = remaining_tables_data_6[0]
australia_fall_of_wickets_list_1 = australia_fall_of_wickets_list[len("Fall of wickets: "):india_fall_of_wickets_list.index("\xa0•\xa0DRS BOWLING")].split("), ")
australia_fall_of_wickets = pd.DataFrame(columns=["Wicket","Run","Batter","Over","Ball"])
fall = []
ctr = 0
for i in australia_fall_of_wickets_list_1:
    wickets_runs = i[:i.index(" (")].split("-")
    australia_fall_of_wickets.loc[ctr,"Wicket"] = wickets_runs[0]
    australia_fall_of_wickets.loc[ctr,"Run"] = wickets_runs[1]
    australia_fall_of_wickets.loc[ctr,"Batter"] = i[i.index(" (")+2:i.index(", ")]
    over = i[i.index(", ")+2:i.index(" ov")].split(".")
    australia_fall_of_wickets.loc[ctr,"Over"] = int(over[0])
    australia_fall_of_wickets.loc[ctr,"Ball"] = int(over[1])
    if australia_fall_of_wickets.loc[ctr,"Ball"] == 6:
        australia_fall_of_wickets.loc[ctr,"Over"] = australia_fall_of_wickets.loc[ctr,"Over"] + 1
        australia_fall_of_wickets.loc[ctr,"Ball"] = 0
    ctr += 1
australia_fall_of_wickets


,Wicket,Run,Batter,Over,Ball
0,1,16,David Warner,1,1
1,2,41,Mitchell Marsh,4,3
2,3,47,Steven Smith,7,0
3,4,239,Travis Head,42,5


In [139]:
australia_total = australia_batting["Runs"].sum() + np.array([i for i in australia_extras.values()]).sum()
australia_total_balls = australia_batting["Balls"].sum()
australia_overs = int(np.floor(australia_total_balls / 6))
australia_balls = australia_total_balls - australia_overs * 6


In [171]:
remaining_tables_data_7 = remaining_tables_data_6[1:]
remaining_tables_data_8 = remaining_tables_data_7[remaining_tables_data_7.index("NB")+1:remaining_tables_data_7.index("See all photos Narendra Modi Stadium, Ahmedabad")+1]
remaining_tables_data_9 = remaining_tables_data_7[remaining_tables_data_7.index("See all photos Narendra Modi Stadium, Ahmedabad"):]

ctr = 0
dts = []
dt = []
ctr = 0
for i in remaining_tables_data_8:
    dt.append(i)
    if i=="See all photos":
        dts.append(dt)
        dt = []
    ctr += 1
dts.append(dt)

india_bowling_list = []
for i in dts[:-1]:
    india_bowling_list.append(i[0:11])

india_bowling_list.append(dts[-1][:dts[-1].index("Mohammed Siraj")])

kk = dts[-1][dts[-1].index("Mohammed Siraj"):-1]
kl = [str(dts[-1][-1].split(" ")[0])]
kk = kk + kl


india_bowling_list.append(kk[0:11])


lk_1 = india_bowling_list[-2][:india_bowling_list[-2].index("Kuldeep Yadav")]
lk_2 = india_bowling_list[-2][india_bowling_list[-2].index("Kuldeep Yadav"):]

india_bowling_list_1 = india_bowling_list.copy()

india_bowling_list = india_bowling_list_1[0:2]
india_bowling_list.append(lk_1)
india_bowling_list.append(lk_2)
india_bowling_list.append(india_bowling_list_1[-1])


india_bowling = pd.DataFrame(india_bowling_list,columns=["Bowler","Overs","Maidens","Runs","Wickets","Economy","Zeros","Fours","Sixes","Wides","No Balls"])
india_bowling


,Bowler,Overs,Maidens,Runs,Wickets,Economy,Zeros,Fours,Sixes,Wides,No Balls
0,Jasprit Bumrah,9,2,43,2,4.77,37,8,0,0,0
1,Mohammed Shami,7,1,47,1,6.71,28,6,1,3,0
2,Ravindra Jadeja,10,0,43,0,4.30,29,1,1,1,0
3,Kuldeep Yadav,10,0,56,0,5.60,25,3,2,0,0
4,Mohammed Siraj,7,0,45,1,6.42,16,4,1,0,0
